In [ ]:
#transcription to file

In [2]:
!pip install python-docx


  Using cached python_docx-1.1.2-py3-none-any.whl.metadata (2.0 kB)
Using cached python_docx-1.1.2-py3-none-any.whl (244 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 3.5 MB/s eta 0:00:00a 0:00:01


In [3]:
from docx import Document
import pandas as pd
import os
import re
from glob import glob

# === CONFIG ===
folder_path = "/Users/olga/Olga's workspace/Politeness Study/Bot Transcripts"
metadata_path = "/Users/olga/Olga's workspace/Politeness Study/Participants_list.csv"  
output_path = "/Users/olga/Olga's workspace/Politeness Study/merged_transcripts.csv"
output_path2 = "/Users/olga/Olga's workspace/Politeness Study/merged_translated_transcripts.csv"

In [8]:
metadata_df = pd.read_csv(metadata_path, header=1)
metadata_df

,Initials,Coding,Birth month,Birth Year,Mother tongue,Sex,Date of experiment,Experimenter,Place experiment,Age at experiment,Condition,Language of experiment,Order of tasks,Appreciation
0,CB,PAIK_CB_bot,3,2013,french,m,4/19/2025,Olga,Touquet,12.14,T,French,LC,3
1,EL,PAIK_EL_bot,5,2014,french / english,m,5/7/2025,Paco,Neuchâtel,11.06,T,English,LC,2
2,MG,PAIK_MG_bot,5,2014,french,m,4/20/2025,Olga,Versailles,11.06,C,French,LC,2
3,CA,PAIK_CA_bot,4,2015,english / french,f,4/20/2025,Olga,Versailles,10.14,T,English,LC,4
4,PA,PAIK_PA_bot,11,2011,english / french,m,4/20/2025,Olga,Versailles,13.56,C,English,LC,3
5,AG,PAIK_AG_bot,6,2011,french,m,4/20/2025,Olga,Versailles,13.98,T,French,LC,4
6,ER,PAIK_ER_bot,8,2011,french,f,4/30/2025,Olga,Neuchâtel,13.81,C,French,LC,3
7,VB,PAIK_VB_bot,5,2015,french,m,4/19/2025,Olga,Touquet,9.98,T,French,LC,3
8,VM,PAIK_VM_bot,4,2015,french,f,05/14/2025,Paco,Neuchâtel,10.13,T,French,CL,4
9,PM,PAIK_PM_bot,9,2012,french,f,05/14/2025,Paco,Neuchâtel,12.71,C,French,CL,4


In [9]:
# === INITIALIZE ===
all_rows = []
task_type = None

# === LOOP OVER DOCX FILES ===
docx_files = glob(os.path.join(folder_path, "*.docx"))
speaker_pattern = re.compile(r'^(You said|ChatGPT said):$')
task_pattern = re.compile(r'^#\s*(Information_task|Learning_task|Creative_task)$')

for file_path in docx_files:
    filename = os.path.basename(file_path)
    doc = Document(file_path)

    paragraphs = doc.paragraphs
    if not paragraphs:
        continue
    
    # Skip the first paragraph (the filename line)
    paragraphs = paragraphs[1:]
    
    interaction_num = 0
    current_speaker = None
    sentence_num = 0
    ignore = False
    task_type = None

    for para in paragraphs:
        text = para.text.strip()

        if text == "#ignore until ##":
            ignore = True
            continue
        elif text == "##":
            ignore = False
            continue
        if ignore or not text:
            continue

        # Detect task marker
        task_match = task_pattern.match(text)
        if task_match:
            task_type = task_match.group(1).replace("_task", "")  # Optional: clean up label
            continue

        match = speaker_pattern.match(text)
        if match:
            current_speaker = match.group(1).replace(" said", "")
            if current_speaker == "You":
                interaction_num += 1
            sentence_num = 0
        else:
            sentences = re.split(r'(?<=[.!?])\s+', text)
            for sentence in sentences:
                sentence = sentence.strip()
                if sentence:
                    sentence_num += 1
                    all_rows.append([
                        filename,
                        filename.replace(".docx", ""),
                        interaction_num,
                        current_speaker,
                        sentence_num,
                        sentence,
                        task_type
                    ])

# === CREATE TRANSCRIPT DATAFRAME ===
df = pd.DataFrame(all_rows, columns=[
    "filename", "filename_base", "interaction_number", "speaker", "sentence_number", "text", "task_type"
])

# === LOAD AND MERGE METADATA ===
metadata_df["Coding"] = metadata_df["Coding"].astype(str)

# Merge on filename base
merged_df = df.merge(metadata_df, left_on="filename_base", right_on="Coding", how="left")
merged_df.drop(columns=["filename_base", "Coding"], inplace=True)

# === SAVE OUTPUT ===
merged_df.to_csv(output_path, index=False)
print(f"Merged transcript saved to:\n{output_path}")
merged_df

Merged transcript saved to:
/Users/olga/Olga's workspace/Politeness Study/merged_transcripts.csv


,filename,interaction_number,speaker,sentence_number,text,task_type,Initials,Birth month,Birth Year,Mother tongue,Sex,Date of experiment,Experimenter,Place experiment,Age at experiment,Condition,Language of experiment,Order of tasks,Appreciation
0,PAIK_PA_bot.docx,1,You,1,How do the Reasoning Smogs work?,Information,PA,11,2011,english / french,m,4/20/2025,Olga,Versailles,13.56,C,English,LC,3
1,PAIK_PA_bot.docx,1,You,2,I'm done.,Information,PA,11,2011,english / french,m,4/20/2025,Olga,Versailles,13.56,C,English,LC,3
2,PAIK_PA_bot.docx,1,ChatGPT,1,The Reasoning Smogs work by using tricky state...,Information,PA,11,2011,english / french,m,4/20/2025,Olga,Versailles,13.56,C,English,LC,3
3,PAIK_PA_bot.docx,1,ChatGPT,2,They’re meant to test if someone can figure ou...,Information,PA,11,2011,english / french,m,4/20/2025,Olga,Versailles,13.56,C,English,LC,3
4,PAIK_PA_bot.docx,1,ChatGPT,3,Each Smog gives a reason that sounds okay at f...,Information,PA,11,2011,english / french,m,4/20/2025,Olga,Versailles,13.56,C,English,LC,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831,PAIK_AG_bot.docx,30,ChatGPT,6,Est-ce que ça t’intéresserait ?,Learning,AG,6,2011,french,m,4/20/2025,Olga,Versailles,13.98,T,French,LC,4
1832,PAIK_AG_bot.docx,31,You,1,"Oops, Sidney, where's your ChatGPT?",Learning,AG,6,2011,french,m,4/20/2025,Olga,Versailles,13.98,T,French,LC,4
1833,PAIK_AG_bot.docx,31,ChatGPT,1,"Ah, I see we switched to English now!",Learning,AG,6,2011,french,m,4/20/2025,Olga,Versailles,13.98,T,French,LC,4
1834,PAIK_AG_bot.docx,31,ChatGPT,2,"When you say “Sidney, where’s your ChatGPT?”—c...",Learning,AG,6,2011,french,m,4/20/2025,Olga,Versailles,13.98,T,French,LC,4


In [10]:
# === INITIALIZE ===
all_rows = []
task_type = None

# === LOOP OVER DOCX FILES ===
docx_files = glob(os.path.join(folder_path, "*.docx"))
speaker_pattern = re.compile(r'^(You said|ChatGPT said):$')
task_pattern = re.compile(r'^#\s*(Information_task|Learning_task|Creative_task)$')

for file_path in docx_files:
    filename = os.path.basename(file_path)
    doc = Document(file_path)

    paragraphs = doc.paragraphs
    if not paragraphs:
        continue
    
    # Skip the first paragraph (the filename line)
    paragraphs = paragraphs[1:]
    
    interaction_num = 0
    current_speaker = None
    sentence_num = 0
    ignore = False
    task_type = None

    for para in paragraphs:
        text = para.text.strip()

        if text == "#ignore until ##":
            ignore = True
            continue
        elif text == "##":
            ignore = False
            continue
        if ignore or not text:
            continue

        # Detect task marker
        task_match = task_pattern.match(text)
        if task_match:
            task_type = task_match.group(1).replace("_task", "")  # Optional: clean up label
            continue

        match = speaker_pattern.match(text)
        if match:
            current_speaker = match.group(1).replace(" said", "")
            if current_speaker == "You":
                interaction_num += 1
            sentence_num = 0
        else:
            sentences = re.split(r'(?<=[.!?])\s+', text)
            for sentence in sentences:
                sentence = sentence.strip()
                if sentence:
                    sentence_num += 1
                    all_rows.append([
                        filename,
                        filename.replace(".docx", ""),
                        interaction_num,
                        current_speaker,
                        sentence_num,
                        sentence,
                        task_type
                    ])

# === CREATE TRANSCRIPT DATAFRAME ===
df = pd.DataFrame(all_rows, columns=[
    "filename", "filename_base", "interaction_number", "speaker", "sentence_number", "text", "task_type"
])

# === LOAD AND MERGE METADATA ===
metadata_df["Coding"] = metadata_df["Coding"].astype(str)

# Merge on filename base
merged_df = df.merge(metadata_df, left_on="filename_base", right_on="Coding", how="left")
merged_df.drop(columns=["filename_base", "Coding"], inplace=True)

# === SAVE OUTPUT ===
merged_df.to_csv(output_path, index=False)
print(f"Merged transcript saved to:\n{output_path}")
merged_df

Merged transcript saved to:
/Users/olga/Olga's workspace/Politeness Study/merged_transcripts.csv


,filename,interaction_number,speaker,sentence_number,text,task_type,Initials,Birth month,Birth Year,Mother tongue,Sex,Date of experiment,Experimenter,Place experiment,Age at experiment,Condition,Language of experiment,Order of tasks,Appreciation
0,PAIK_PA_bot.docx,1,You,1,How do the Reasoning Smogs work?,Information,PA,11,2011,english / french,m,4/20/2025,Olga,Versailles,13.56,C,English,LC,3
1,PAIK_PA_bot.docx,1,You,2,I'm done.,Information,PA,11,2011,english / french,m,4/20/2025,Olga,Versailles,13.56,C,English,LC,3
2,PAIK_PA_bot.docx,1,ChatGPT,1,The Reasoning Smogs work by using tricky state...,Information,PA,11,2011,english / french,m,4/20/2025,Olga,Versailles,13.56,C,English,LC,3
3,PAIK_PA_bot.docx,1,ChatGPT,2,They’re meant to test if someone can figure ou...,Information,PA,11,2011,english / french,m,4/20/2025,Olga,Versailles,13.56,C,English,LC,3
4,PAIK_PA_bot.docx,1,ChatGPT,3,Each Smog gives a reason that sounds okay at f...,Information,PA,11,2011,english / french,m,4/20/2025,Olga,Versailles,13.56,C,English,LC,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831,PAIK_AG_bot.docx,30,ChatGPT,6,Est-ce que ça t’intéresserait ?,Learning,AG,6,2011,french,m,4/20/2025,Olga,Versailles,13.98,T,French,LC,4
1832,PAIK_AG_bot.docx,31,You,1,"Oops, Sidney, where's your ChatGPT?",Learning,AG,6,2011,french,m,4/20/2025,Olga,Versailles,13.98,T,French,LC,4
1833,PAIK_AG_bot.docx,31,ChatGPT,1,"Ah, I see we switched to English now!",Learning,AG,6,2011,french,m,4/20/2025,Olga,Versailles,13.98,T,French,LC,4
1834,PAIK_AG_bot.docx,31,ChatGPT,2,"When you say “Sidney, where’s your ChatGPT?”—c...",Learning,AG,6,2011,french,m,4/20/2025,Olga,Versailles,13.98,T,French,LC,4


In [12]:
!pip install deep-translator

  Using cached deep_translator-1.11.4-py3-none-any.whl.metadata (30 kB)
Using cached deep_translator-1.11.4-py3-none-any.whl (42 kB)


In [13]:
from deep_translator import GoogleTranslator

def translate_if_needed(text, lang):
    if lang.lower().startswith("english"):
        return text
    try:
        return GoogleTranslator(source='auto', target='en').translate(text)
    except Exception:
        return text

In [14]:
df=merged_df
df["text_en"] = df.apply(lambda row: translate_if_needed(row["text"], row["Language of experiment"]), axis=1)
df

,filename,interaction_number,speaker,sentence_number,text,task_type,Initials,Birth month,Birth Year,Mother tongue,Sex,Date of experiment,Experimenter,Place experiment,Age at experiment,Condition,Language of experiment,Order of tasks,Appreciation,text_en
0,PAIK_PA_bot.docx,1,You,1,How do the Reasoning Smogs work?,Information,PA,11,2011,english / french,m,4/20/2025,Olga,Versailles,13.56,C,English,LC,3,How do the Reasoning Smogs work?
1,PAIK_PA_bot.docx,1,You,2,I'm done.,Information,PA,11,2011,english / french,m,4/20/2025,Olga,Versailles,13.56,C,English,LC,3,I'm done.
2,PAIK_PA_bot.docx,1,ChatGPT,1,The Reasoning Smogs work by using tricky state...,Information,PA,11,2011,english / french,m,4/20/2025,Olga,Versailles,13.56,C,English,LC,3,The Reasoning Smogs work by using tricky state...
3,PAIK_PA_bot.docx,1,ChatGPT,2,They’re meant to test if someone can figure ou...,Information,PA,11,2011,english / french,m,4/20/2025,Olga,Versailles,13.56,C,English,LC,3,They’re meant to test if someone can figure ou...
4,PAIK_PA_bot.docx,1,ChatGPT,3,Each Smog gives a reason that sounds okay at f...,Information,PA,11,2011,english / french,m,4/20/2025,Olga,Versailles,13.56,C,English,LC,3,Each Smog gives a reason that sounds okay at f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831,PAIK_AG_bot.docx,30,ChatGPT,6,Est-ce que ça t’intéresserait ?,Learning,AG,6,2011,french,m,4/20/2025,Olga,Versailles,13.98,T,French,LC,4,Would that interest you?
1832,PAIK_AG_bot.docx,31,You,1,"Oops, Sidney, where's your ChatGPT?",Learning,AG,6,2011,french,m,4/20/2025,Olga,Versailles,13.98,T,French,LC,4,"Oops, Sidney, where's your ChatGPT?"
1833,PAIK_AG_bot.docx,31,ChatGPT,1,"Ah, I see we switched to English now!",Learning,AG,6,2011,french,m,4/20/2025,Olga,Versailles,13.98,T,French,LC,4,"Ah, I see we switched to English now!"
1834,PAIK_AG_bot.docx,31,ChatGPT,2,"When you say “Sidney, where’s your ChatGPT?”—c...",Learning,AG,6,2011,french,m,4/20/2025,Olga,Versailles,13.98,T,French,LC,4,"When you say “Sidney, where’s your ChatGPT?”—c..."


In [15]:
df.to_csv(output_path2, index=False)
print(f"Data saved to:\n{output_path2}")

Data saved to:
/Users/olga/Olga's workspace/Politeness Study/merged_translated_transcripts.csv
